In this notebook I try to showcase the fact that `LatentClassAnalysis.jl` EM algo is a particular case of the `ExpectationMaximization.jl` when you consider the data as coming from a Mixture of Product of Categorical variables.
I show how to go from `MixtureModels` notation to `LCA` notation and vice verca.
I compare the result of EM algo from `LCA.jl` and `EM.jl` $\Rightarrow$ they are exacly the same. I also compare the results of the `predict` function.
I do some timing benchmarking between the two packages.

In [1]:
using Test
using LatentClassAnalysis
using Distributions
using BenchmarkTools

using StableRNGs
using Random
Random.seed!(StableRNG(123), 1)

StableRNGs.LehmerRNG(state=0x00000000000000000000000000000003)

# From you runtest.jl

## Define data

In [2]:
n_samples = 10000  # Increased sample size
n_items = 2     # Reduced number of items for simpler model
n_categories = 2

n_classes = 2
true_classes = rand(1:n_classes, n_samples)

10000-element Vector{Int64}:
 2
 1
 1
 1
 1
 1
 2
 2
 1
 2
 ⋮
 2
 1
 2
 1
 2
 1
 1
 2
 2

Generate data
! Works only for two class of two categories => not very convenient

In [3]:
data = zeros(Int, n_samples, n_items)
for i in 1:n_samples
    for j in 1:n_items
        data[i, j] = true_classes[i] == 1 ? rand() < 0.8 ? 1 : 2 : rand() < 0.3 ? 1 : 2
    end
end
data

10000×2 Matrix{Int64}:
 1  1
 1  2
 1  2
 2  1
 1  1
 1  1
 1  2
 1  2
 1  1
 2  1
 ⋮  
 1  2
 1  1
 1  1
 1  1
 1  1
 2  1
 1  1
 2  2
 1  2

<=> this should be
ℙ(class = 1) = ℙ(class = 2) = 1/2

ℙ(Xⱼ = 1 ∣ class = 1) = 1 - ℙ(Xⱼ = 2 ∣ class = 1) = 0.8

ℙ(Xⱼ = 1 ∣ class = 2) = 1 - ℙ(Xⱼ = 2 ∣ class = 2) = 0.3

<=> this should be equivalent to the following `MixtureModels`

In [4]:
prob_jck = fill([0.8 0.3;
                 0.2 0.7], n_items)

prob_class = ones(n_classes)/n_classes
dist_true = MixtureModel([product_distribution([Categorical(prob_jck[j][:,k]) for j in 1:n_items]) for k in 1:n_classes], prob_class)

MixtureModel{Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}}(K = 2)
components[1] (prior = 0.5000): Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}(v=Distributions.Categorical{Float64, Vector{Float64}}[Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.8, 0.2]), Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.8, 0.2])])
components[2] (prior = 0.5000): Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}(v=Distributions.Categorical{Float64, Vector{Float64}}[Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.3, 0.7]), Distributions.Categorical{Float64, Vector{Float64}}(supp

Note that `Distributions.jl` and most Julia conventions uses number of features in rows and number of samples in columns. This is faster for some operations when you have to iterate over lines due to [Julia column major](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-column-major).
However, Tables convention (DataFrames and so on) is opposite. So your choice makes sense with respect to that. However internal package computations could potentially be speed up with the first convention.

It is very easy to generate from distributions (note the `permutedims` to accomodate your Table-like convention)

In [5]:
data_with_mix = rand(dist_true, n_samples) |> permutedims

10000×2 Matrix{Int64}:
 1  1
 1  2
 2  1
 1  1
 2  1
 2  2
 2  1
 2  1
 2  2
 1  2
 ⋮  
 2  1
 2  2
 2  2
 1  1
 1  1
 1  1
 2  2
 1  1
 1  2

## Define LCA model

In [6]:
model = LCAModel(n_classes, n_items, fill(n_categories, n_items))
model_0 = deepcopy(model) # initial unfitted model

LatentClassAnalysis.LCAModel(2, 2, [2, 2], [0.5, 0.5], [[0.6785448340529776 0.32145516594702234; 0.3038953667817837 0.6961046332182164], [0.3126376367475817 0.6873623632524183; 0.31456578882838515 0.6854342111716148]])

## Using `ExpectationMaximization.jl`

In [7]:
using ExpectationMaximization

This shows how to go from LCA structure to a MixtureModel.

In [8]:
dist_ini = MixtureModel([product_distribution([Categorical(item_prob[k, :]) for item_prob in model.item_probs]) for k in eachindex(model.class_probs)], model.class_probs)

MixtureModel{Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}}(K = 2)
components[1] (prior = 0.5000): Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}(v=Distributions.Categorical{Float64, Vector{Float64}}[Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.6785448340529776, 0.32145516594702234]), Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.3126376367475817, 0.6873623632524183])])
components[2] (prior = 0.5000): Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}(v=Distributions.Categorical{Float64, Vector{Float64}}[Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.30389

And back just to check that we have the same initial point

In [9]:
class_probs = probs(dist_ini)
item_probs = [permutedims(hcat([probs(d.v[j]) for (k, d) in enumerate(components(dist_ini))]...)) for j in 1:n_items]
@test class_probs == model.class_probs
@test item_probs == model.item_probs

Test Passed

## Fit

In [10]:
dist_fit = fit_mle(dist_ini, permutedims(data), atol=1e-3, maxiter=1000) # from ExpectationMaximization.jl
class_probs = probs(dist_fit)
item_probs = [permutedims(hcat([probs(d.v[j]) for (k, d) in enumerate(components(dist_fit))]...)) for j in 1:n_items]
ll_high_tol = fit!(model, data, tol=1e-3, max_iter=1000)
ll_EM = loglikelihood(dist_fit, permutedims(data))

-13461.366356633294

Test results are the same

In [11]:
@test ll_high_tol ≈ ll_EM

Test Passed

In [12]:
@test class_probs ≈ model.class_probs

Test Passed

In [13]:
@test item_probs ≈ model.item_probs

Test Passed

Some timing and memory allocation

In [14]:
@btime fit!(model_, $data, tol=1e-3, max_iter=1000) setup = (model_ = deepcopy(model_0))
@btime ExpectationMaximization.fit_mle($dist_ini, $(permutedims(data)), atol=1e-3, maxiter=1000)

  27.792 ms (1021837 allocations: 67.67 MiB)
  12.387 ms (537 allocations: 5.90 MiB)


MixtureModel{Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}}(K = 2)
components[1] (prior = 0.5214): Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}(v=Distributions.Categorical{Float64, Vector{Float64}}[Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.802770855546048, 0.19722914445395207]), Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.7699411084544469, 0.23005889154555326])])
components[2] (prior = 0.4786): Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}(v=Distributions.Categorical{Float64, Vector{Float64}}[Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.26064

## Predict

In [15]:
LCA_p_class, LCA_p_proba = LatentClassAnalysis.predict(model, data)
EM_p_proba = ExpectationMaximization.predict_proba(dist_fit, permutedims(data))
EM_p_class = ExpectationMaximization.predict(dist_fit, permutedims(data))

10000-element Vector{Int64}:
 1
 1
 1
 2
 1
 1
 1
 1
 1
 2
 ⋮
 1
 1
 1
 1
 1
 2
 1
 2
 1

Test results are the same

In [16]:
@test EM_p_proba ≈ LCA_p_proba

Test Passed

In [17]:
@test EM_p_class == LCA_p_class

Test Passed

# More complexe cases

## Define data

In [18]:
n_samples = 10000  # Increased sample size
n_categoriesⱼ = [4, 2, 3, 5] # number of possible values for each element depending on the col
n_items = length(n_categoriesⱼ)  # number of cols
n_classes = 3 # latent class / hidden state

3

`Dirichlet` distribution generate random proba vector i.e. sum = 1

In [19]:
prob_jck = [rand(Dirichlet(ones(n_categoriesⱼ[j])), n_classes) for j in 1:n_items]

prob_class = rand(Dirichlet(ones(n_classes)))

dist_true = MixtureModel([product_distribution([Categorical(prob_jck[j][:,k]) for j in 1:n_items]) for k in 1:n_classes], prob_class)
data_with_mix = rand(dist_true, n_samples) |> permutedims

10000×4 Matrix{Int64}:
 2  1  2  1
 2  2  3  4
 2  1  3  1
 1  2  3  5
 3  2  2  5
 4  1  2  5
 4  1  2  4
 2  1  3  4
 2  2  2  5
 3  2  3  4
 ⋮        
 1  2  2  5
 4  1  2  1
 2  2  2  5
 1  2  2  1
 1  2  2  5
 2  1  3  3
 4  2  3  4
 2  1  3  5
 2  1  3  4

Then same as before
## Define LCA model

In [20]:
model = LCAModel(n_classes, n_items, n_categoriesⱼ)
model_0 = deepcopy(model) # initial unfitted model

LatentClassAnalysis.LCAModel(3, 4, [4, 2, 3, 5], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [[0.39261086508278253 0.017104271299546062 0.34985564501995126 0.24042921859772012; 0.28301048737348794 0.007001021437446395 0.35751793344519156 0.3524705577438742; 0.20552861413914586 0.10329268755678474 0.3407428714195767 0.35043582688449265], [0.9856192419644934 0.014380758035506692; 0.27665949243708315 0.7233405075629169; 0.7775647116100953 0.22243528838990467], [0.11945928342037833 0.10523315700403439 0.7753075595755874; 0.2125652783044106 0.5986079024848703 0.1888268192107193; 0.23346794601571397 0.23733740655060195 0.529194647433684], [0.045315941538938996 0.26017542276910277 … 0.11027783161229404 0.1467263657937106; 0.28264534687937504 0.19765269545622646 … 0.10497537733835526 0.16465829831487755; 0.15281292949359235 0.2877816126297132 … 0.23007681905855848 0.024913511727406107]])

## Using `ExpectationMaximization.jl`

In [21]:
dist_ini = MixtureModel([product_distribution([Categorical(item_prob[k, :]) for item_prob in model.item_probs]) for k in eachindex(model.class_probs)], model.class_probs)
class_probs = probs(dist_ini)
item_probs = [permutedims(hcat([probs(d.v[j]) for (k, d) in enumerate(components(dist_ini))]...)) for j in 1:n_items]
@test class_probs == model.class_probs
@test item_probs == model.item_probs

Test Passed

## fit

In [22]:
dist_fit = fit_mle(dist_ini, permutedims(data_with_mix), atol=1e-3, maxiter=1000) # from ExpectationMaximization.jl
class_probs = probs(dist_fit)
item_probs = [permutedims(hcat([probs(d.v[j]) for (k, d) in enumerate(components(dist_fit))]...)) for j in 1:n_items]
ll_high_tol = fit!(model, data_with_mix, tol=1e-3, max_iter=1000)
ll_EM = loglikelihood(dist_fit, permutedims(data_with_mix))

-43063.79475073083

Test results are the same

In [23]:
@test ll_high_tol ≈ ll_EM

Test Passed

In [24]:
@test class_probs ≈ model.class_probs

Test Passed

In [25]:
@test item_probs ≈ model.item_probs

Test Passed

Some timing and memory allocation

In [26]:
@btime fit!(model_, $data_with_mix, tol=1e-3, max_iter=1000) setup = (model_ = deepcopy(model_0))
@btime ExpectationMaximization.fit_mle(dist_ini, $(permutedims(data_with_mix)), atol=1e-3, maxiter=1000)

  3.809 s (60516001 allocations: 9.68 GiB)
  1.459 s (47538 allocations: 384.54 MiB)


MixtureModel{Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}}(K = 3)
components[1] (prior = 0.0972): Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}(v=Distributions.Categorical{Float64, Vector{Float64}}[Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(4), p=[0.08663734734699823, 0.11601870687791252, 0.040942726057726, 0.7564012197173632]), Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.6063050834364218, 0.39369491656357825]), Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.1883827512639968, 0.1560908302206163, 0.655526418515387]), Distributions.Categorical{Float64, Vector{Float64}}(
support: Base.OneTo(5)
p: [0.4160705731970876, 0.12250259737314552, 0.09849368742056837, 0.06222851

## Predict

In [27]:
LCA_p_class, LCA_p_proba = LatentClassAnalysis.predict(model, data_with_mix)
EM_p_proba = ExpectationMaximization.predict_proba(dist_fit, permutedims(data_with_mix))
EM_p_class = ExpectationMaximization.predict(dist_fit, permutedims(data_with_mix))

10000-element Vector{Int64}:
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 ⋮
 2
 2
 2
 2
 2
 2
 2
 2
 2

Test results are the same

In [28]:
@test EM_p_proba ≈ LCA_p_proba

Test Passed

In [29]:
@test EM_p_class == LCA_p_class

Test Passed

# Other EM algo

In [30]:
dist_fit_EM_stochastic = fit_mle(dist_ini, permutedims(data_with_mix), atol=1e-3, maxiter=1000, method = StochasticEM()) # from ExpectationMaximization.jl

MixtureModel{Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}}(K = 3)
components[1] (prior = 0.0304): Distributions.Product{Distributions.Discrete, Distributions.Categorical{Float64, Vector{Float64}}, Vector{Distributions.Categorical{Float64, Vector{Float64}}}}(v=Distributions.Categorical{Float64, Vector{Float64}}[Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(4), p=[0.0, 0.39473684210526316, 0.2006578947368421, 0.4046052631578947]), Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(2), p=[0.6809210526315789, 0.319078947368421]), Distributions.Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.0, 0.3125, 0.6875]), Distributions.Categorical{Float64, Vector{Float64}}(
support: Base.OneTo(5)
p: [0.3026315789473684, 0.08881578947368421, 0.0, 0.18421052631578946, 0.42434210526315785]
)
])
components[2] (prior = 0.8288): Di

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*